<a href="https://colab.research.google.com/github/chandutr/Natural-Language-Toolkit-Projects/blob/main/sms_spam_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df=pd.read_csv(r'/content/spam.csv', encoding='latin-1')
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [ ]:
df = df.rename(columns={'v1': 'labels', 'v2':'text'})
df.head()

,labels,text,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)
df

,labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
#Text preprocessing
#create a function to do normalization(for removing punctuation and convert to lower case)
import re
def remove_punctuation_lower(text):
  text=re.sub(r'[^a-zA-Z0-9\s]',' ',text)
  text=text.lower()
  return text

df['text']=df['text'].apply(remove_punctuation_lower)

In [ ]:
df.head()

,labels,text
0,ham,go until jurong point crazy available only ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives aro...


In [ ]:
#Function to remove stop words
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize # Import word_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')

stop_words=set(stopwords.words('english'))

def remove_stopwords(text):
  # Use word_tokenize instead of split()
  tokens = word_tokenize(text)
  filtered_tokens=[word for word in tokens if word not in stop_words]
  return ' '.join(filtered_tokens)

df['text']=df['text'].apply(remove_stopwords)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df.head()

,labels,text
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think goes usf lives around though


In [ ]:
# Stemming
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def stem_text(text):
  """Applies stemming to a string."""
  tokens = text.split() # Assuming text is already tokenized by spaces after stopword removal
  stemmed_tokens = [stemmer.stem(word) for word in tokens]
  return ' '.join(stemmed_tokens)

df['text'] = df['text'].apply(stem_text)

In [ ]:
df.head()

,labels,text
0,ham,go jurong point crazi avail bugi n great world...
1,ham,ok lar joke wif u oni
2,spam,free entri 2 wkli comp win fa cup final tkt 21...
3,ham,u dun say earli hor u c alreadi say
4,ham,nah think goe usf live around though


In [ ]:
!pip install gensim

In [ ]:
import numpy as np
from gensim.models import Word2Vec

In [ ]:
w2v_model=Word2Vec(sentences=df['text'], vector_size=100, window=5, min_count=1)
w2v_model

vector_size=100: Each word will be represented by a 100D array.

In [ ]:
#convert sentences to averaged Word2Vec vectors, then only we can apply ML Algo
def get_avg_w2v(tokens, model, vector_size):
  vec=np.zeros(vector_size) #will store the sum of all word vector for a sentence
  count=0 #used to count how many words were found in the word2vec vocabulary
  for word in tokens:
    if word in model.wv:
      vec+=model.wv[word]
      count+=1
  return vec/count if count!=0 else vec #if atleast one word got matched => return the avg of all vectors
  #else return the original zero vector

df['w2v_vector']=df['text'].apply(lambda x: get_avg_w2v(x, w2v_model, 100))
df.head()

,labels,text,w2v_vector
0,ham,go jurong point crazi avail bugi n great world...,"[-0.01622709774069096, 0.11012057415025897, 0...."
1,ham,ok lar joke wif u oni,"[-0.02773955464363098, 0.1075933047880729, 0.0..."
2,spam,free entri 2 wkli comp win fa cup final tkt 21...,"[0.12250312389523695, -0.011576692845361928, -..."
3,ham,u dun say earli hor u c alreadi say,"[-0.027678324707916806, 0.14204411639698913, 0..."
4,ham,nah think goe usf live around though,"[-0.04687518812716007, 0.13687183162740743, 0...."


In [ ]:
df['w2v_vector'][0].shape

(100,)

In [ ]:
type(df['w2v_vector'][0])

numpy.ndarray

In [ ]:
df['label_encoded']=df['labels'].map({'ham':0,'spam':1})

In [ ]:
x=np.array(df['w2v_vector'].tolist())
y=df['label_encoded'].values

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
y_pred=model.predict(x_test)
y_pred

array([0, 0, 1, ..., 0, 0, 1])

In [ ]:
from sklearn.metrics import accuracy_score,classification_report
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9668161434977578
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       965
           1       0.92      0.83      0.87       150

    accuracy                           0.97      1115
   macro avg       0.95      0.91      0.93      1115
weighted avg       0.97      0.97      0.97      1115

